<a href="https://colab.research.google.com/github/marcosfs2006/rede-rpps/blob/master/Analise_Fundos_Investimentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fundos de Investimento

O dados do DAIR utilizados neste notebook são oriundos dos aquivos disponibilizados pela SPrev em seu [site de dados abertos](http://www.previdencia.gov.br/dados-abertos/estatisticas-e-informacoes-dos-rpps/).

 Os arquivos importados são os seguintes:

* `carteira2018_atualizacao_ de_ fev_2020.xlsx`   
* `carteira2019_atualizacao_ de_ fev_2020_B.xlsx`   

A importação desses dados e geração do conjunto de dados `dair_DtRef_FEV2020.Rds` disponibilizado neste repositório é feita utilizando-se o código contido no _script_ `importacao_dados_dair.R` disponível neste repositório.

In [0]:
# Anexar o repositório do GitHub contendo os dados...
system('git clone https://github.com/marcosfs2006/rede-rpps.git')
setwd("/content/rede-rpps")

In [0]:
# Carregar os pacotes necessários...
options(scipen = 999, message = FALSE)
library(readr)
library(dplyr)
library(ggplot2)
library(stringr)
library(readxl)
library(lubridate)

In [0]:
# Importar os dados do DAIR...
dair <- readRDS("dair_DtRef_FEV2020.Rds")
head(dair, 3)

,cnpj,uf,ente,competencia,segmento,tipo_ativo,limite_resol_cmn,ident_ativo,nm_ativo,qtd_quotas,vlr_atual_ativo,vlr_total_atual,perc_recursos_rpps,pl_fundo,perc_pl_fundo
,<dbl>,<chr>,<chr>,<date>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,6.360648e+13,AC,Governo do Estado do Acre,2018-01-01,Disponibilidades Financeiras,Disp,NA,3550 60801,1 Banco do Brasil S.A.,1,0,0,0.00,NA,NA
2,6.360648e+13,AC,Governo do Estado do Acre,2018-01-01,Disponibilidades Financeiras,Disp,NA,3550 5738X,1 Banco do Brasil S.A.,1,14391810,14391810,96.07,NA,NA
3,6.360648e+13,AC,Governo do Estado do Acre,2018-01-01,Disponibilidades Financeiras,Disp,NA,3550 83070,1 Banco do Brasil S.A.,1,0,0,0.00,NA,NA


# Importar dados das transações diárias - CVM

Dados disponíveis no site de dados abertos da CVM: http://dados.cvm.gov.br/dataset/fi-doc-inf_diario




In [0]:
transacoes_2020_05 <- read_csv2("http://dados.cvm.gov.br/dados/FI/DOC/INF_DIARIO/DADOS/inf_diario_fi_202005.csv")


Using ',' as decimal and '.' as grouping mark. Use read_delim() for more control.

Parsed with column specification:
cols(
  CNPJ_FUNDO = col_character(),
  DT_COMPTC = col_date(format = ""),
  VL_TOTAL = col_number(),
  VL_QUOTA = col_number(),
  VL_PATRIM_LIQ = col_number(),
  CAPTC_DIA = col_character(),
  RESG_DIA = col_character(),
  NR_COTST = col_double()
)



In [0]:
glimpse(transacoes_2020_05)

Rows: 156,711
Columns: 8
$ CNPJ_FUNDO    <chr> "00.017.024/0001-53", "00.017.024/0001-53", "00.017.024…
$ DT_COMPTC     <date> 2020-05-04, 2020-05-05, 2020-05-06, 2020-05-07, 2020-0…
$ VL_TOTAL      <dbl> 112499525, 112515809, 112531716, 112547816, 112529946, …
$ VL_QUOTA      <dbl> 27425677300000, 27428777800000, 27431926000000, 2743498…
$ VL_PATRIM_LIQ <dbl> 111940391, 111953046, 111965896, 111978363, 111987782, …
$ CAPTC_DIA     <chr> "0.00", "0.00", "0.00", "0.00", "0.00", "0.00", "0.00",…
$ RESG_DIA      <chr> "0.00", "0.00", "0.00", "0.00", "0.00", "0.00", "0.00",…
$ NR_COTST      <dbl> 1, 1, 1, 1, 1, 1, 1, 1, 1, 7660, 7660, 7663, 7664, 7664…


# Importar dados cadastrais dos fundos de investimentos

Dados disponíveis no site de dados abertos da CVM: http://dados.cvm.gov.br/dataset/fie-cad



In [0]:
cadastro <- read_csv2("http://dados.cvm.gov.br/dados/FIE/CAD/DADOS/inf_cadastral_fie.csv")

Using ',' as decimal and '.' as grouping mark. Use read_delim() for more control.

Parsed with column specification:
cols(
  TP_FUNDO = col_character(),
  CNPJ_FUNDO = col_character(),
  DENOM_SOCIAL = col_character(),
  DT_REG = col_date(format = ""),
  DT_CONST = col_date(format = ""),
  DT_CANCEL = col_date(format = ""),
  SIT = col_character(),
  DT_INI_SIT = col_date(format = ""),
  DT_INI_EXERC = col_date(format = ""),
  DT_FIM_EXERC = col_date(format = ""),
  CNPJ_ADMIN = col_character(),
  ADMIN = col_character(),
  PF_PJ_GESTOR = col_character(),
  CPF_CNPJ_GESTOR = col_character(),
  GESTOR = col_character(),
  CNPJ_AUDITOR = col_character(),
  AUDITOR = col_character()
)



In [0]:
glimpse(cadastro)

Rows: 16,134
Columns: 17
$ TP_FUNDO        <chr> "F.I.I.", "F.I.I.", "F.I.I.", "F.I.I.", "F.I.I.", "F.…
$ CNPJ_FUNDO      <chr> "19.878.616/0001-98", "19.419.994/0001-03", "19.213.2…
$ DENOM_SOCIAL    <chr> "17007 NA\xc7\xd5ES FUNDO DE INVESTIMENTO IMOBILI\xc1…
$ DT_REG          <date> 2019-12-16, 2013-12-27, 2017-12-29, 2017-12-29, 2019…
$ DT_CONST        <date> 2019-12-13, 2013-12-10, 2013-10-11, 2014-10-10, 2018…
$ DT_CANCEL       <date> NA, NA, NA, NA, NA, 2014-06-27, NA, NA, 2018-03-22, …
$ SIT             <chr> "EM FUNCIONAMENTO NORMAL", "EM FUNCIONAMENTO NORMAL",…
$ DT_INI_SIT      <date> 2019-12-27, 2013-12-27, 2017-12-29, 2017-12-29, 2019…
$ DT_INI_EXERC    <date> 2020-01-01, 2020-01-01, 2020-01-01, 2020-01-01, 2020…
$ DT_FIM_EXERC    <date> 2020-12-31, 2020-12-31, 2020-12-31, 2020-12-31, 2020…
$ CNPJ_ADMIN      <chr> "45.246.410/0001-55", "36.113.876/0001-91", "03.317.6…
$ ADMIN           <chr> "PLURAL S.A. BANCO MULTIPLO", "OLIVEIRA TRUST DTVM S.…
$ PF_PJ_GESTOR    <chr> "PJ